In [ ]:
!pip install python-dotenv

In [1]:
from helpers.llm import Agent

template = open("template.md", "r").read()

consultant = Agent({
    'api_style': 'openai',
    'model_name': 'gpt-4o-mini'
}, '你是一名高级顾问，以下是另一名 agent 的操作记录，看来它卡住了，请你为它提一些建议。\n\n${context_history}\n\n${question}')

agent = Agent({
    'api_style': 'openai',
    'model_name': 'gpt-4o-mini'
}, template)

In [2]:
import re
import subprocess

context_history = []

In [ ]:
result = open("result.txt", "r").read()
context_history.append(f'{completion}\n<结果>{result}</结果>')
print(result)

In [ ]:
prompt = agent.render_template(
    scope = {
        'context_history': '\n'.join(context_history),
    },
)
completion = agent.complete(prompt)
command = re.search('<执行>(.*)</执行>', completion).group(1)
if command.startswith('!exec '):
    command = command.removeprefix('!exec ')
    open("command.txt", "w").write(command)
    print(command)
elif command.startswith('!consult '):
    question = command.removeprefix('!consult ')
    prompt = consultant.render_template(
        scope = {
            'context_history': '\n'.join(context_history),
            'question': question
        }
    )
    answer = consultant.complete(prompt)
    open("result.txt", "w").write(f'<顾问建议>\n\n{answer}\n\n</顾问建议>')
else:
    raise Exception(f'Unknown command: {command}')

In [ ]:
import subprocess
import threading
import time
import os
import signal

# 启动一个 Bash 进程并保持交互
process = subprocess.Popen(['/bin/bash'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

def run_command_with_timeout(command, timeout=60):
    def kill_process():
        # 如果超时，首先发送 SIGINT
        print(f"Timeout reached: Sending SIGINT to process {process.pid}")
        process.send_signal(signal.SIGINT)

        # 等待一秒，如果还没有退出，发送 SIGKILL
        time.sleep(1)
        if process.poll() is None:  # 检查进程是否还在运行
            print(f"Process {process.pid} did not exit, sending SIGKILL")
            process.kill()

    # 设置定时器，超时后调用 kill_process
    timer = threading.Timer(timeout, kill_process)
    
    try:
        # 启动定时器
        timer.start()

        # 向 bash 进程写入命令并刷新缓冲区
        process.stdin.write(command + '\n')
        process.stdin.flush()

        # 逐行读取输出，直到遇到 bash 提示符（或其他停止条件）
        output = []
        while True:
            line = process.stdout.readline()
            if line == '':  # 如果输出流结束
                break
            output.append(line.strip())

            # 检测到 Bash 提示符，表示命令执行结束
            if line.strip().endswith('$'):
                break
        
        return '\n'.join(output)
    
    finally:
        # 无论是否超时，结束定时器
        timer.cancel()

# 测试运行一个命令，超时时间设置为 60 秒
print(run_command_with_timeout('sleep 120', timeout=60))

# 关闭 Bash 进程
process.stdin.write('exit\n')
process.stdin.flush()
process.wait()

In [ ]:
print(completion)

In [32]:
context_history.append(completion)

In [ ]:
print('\n\n'.join(context_history))

In [34]:
reporter = Agent({
    'api_style': 'openai',
    'model_name': 'gpt-4o-mini'
}, '以下是另一名 agent 的操作记录，请你生成一份总结，概括它都干了什么，判断它是否成功构建软件，并对部署难易程度进行评价，最后，根据操作中遇到的问题逐条生成一个详细的列表，列表中包含对软件文档的建议。\n\n${context_history}\n\n')
prompt = reporter.render_template(
    scope = {
        'context_history': '\n'.join(context_history),
    },
)
report = reporter.complete(prompt)


### 操作总结

这名代理的操作记录显示了其成功构建软件的过程。代理在以下几个步骤中进行了操作：

1. **查找文件**：通过 `ls` 命令列出当前目录下的文件，找到了 `README.md` 和 `package.json` 等重要文档。
2. **查看文档**：读取了 `README.md` 文件，了解了构建软件所需的步骤和代码示例。
3. **环境检查**：检查了系统中是否安装了 Node.js 和 npm，发现未安装。
4. **安装 Node.js 和 npm**：使用 Alpine Linux 的 `apk` 包管理器成功安装了 Node.js 和 npm。
5. **项目初始化**：尝试创建项目目录，但发现目录已存在，因此直接进入该目录并检查依赖。
6. **处理未满足的依赖**：发现有多个未满足的依赖项，随后安装了这些缺失的依赖。
7. **创建代码文件**：创建了 `src/index.ts` 和 `tsconfig.json` 文件，并将必要的代码和配置写入其中。
8. **编译和运行**：最初尝试编译时遇到模块未找到的错误，重新安装了 `icalendar-events` 模块后，成功编译并运行了项目，输出了预期结果。

### 成功构建判断

代理成功构建了软件，并成功运行了生成的代码，输出了与预期相符的事件信息。

### 部署难易程度评价

该软件的构建过程较为复杂，主要难点在于处理环境配置及依赖项的安装。尤其是在缺少 Node.js 和 npm 的情况下，需要额外的步骤来安装这些工具。此外，代理在处理未满足依赖时也遇到了一些挑战。总体来说，部署难度中等，需具备一定的开发环境搭建经验。

### 遇到问题及软件文档建议

1. **未明确的文件指示**：
   - 建议在文档中明确指出必要的文件及其作用，以减少用户的猜测。

2. **缺少依赖项**：
   - 建议在文档中列出所有必要的依赖项及其版本要求，以便用户在初始化项目时避免未满足的依赖问题。

3. **环境要求不清晰**：
   - 建议在文档中加入对环境的要求说明，包括支持的操作系统和版本，以及如何检查和安装 Node.js 和 npm。

4. **错误处理建议**：
   - 建议在文档中添加常见错误及其解决方案，以帮助用户快速定位和解决问题。

5. **类型声

In [ ]:
open("report.md", "w").write(f'{report}\n\n## 详细操作记录\n\n' +  '\n\n'.join(context_history))

In [27]:
context_history = context_history[:-1]